In [1]:
import numpy as np
np.random.seed(9527)

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def get_normal_reading(noise_scale=0.1):
    t = np.linspace(0, 10.0, 1001)
    signal = np.sin(2*np.pi*t) + np.sin(4*np.pi*t)
    
    noise = np.random.normal(0, noise_scale, len(t))
    reading = signal + noise
    
    return reading

def get_abnormal_reading(noise_scale=0.1):
    t = np.linspace(0, 10.0, 1001)
    signal = np.sin(2*np.pi*t) + np.sin(4*np.pi*t) + np.sin(10*np.pi*t)
    
    noise = np.random.normal(0, noise_scale, len(t))
    reading = signal + noise
    
    return reading

In [3]:
def get_normal_data(sample_size=1024, sequence_length=100, noise_scale=0.1):
    '''
    x contains the sequences draw from the normal readings
    y contains the label of if the sequence is an anomaly. In this case, all labels in y are 0.
    '''
    reading = get_normal_reading(noise_scale=noise_scale)
      
    x = []
    y = []
    for _ in range(sample_size):
        i = np.random.randint(0, len(reading)-sequence_length-1)
        x.append(reading[i:i+sequence_length])
        y.append(0)
        
    x = np.asarray(x, dtype=np.float32)
    y = np.asarray(y, dtype=np.float32)
    
    return x, y

In [4]:
def get_mixed_data(sample_size=1024, sequence_length=100, noise_scale=0.1, abnormal_ratio=0.01):
    '''
    x contains the sequences draw from the normal/abnormal readings.
    y contains the label of if the sequence is an anomaly. y=1 means the sequence is abnormal.
    '''
    normal_reading = get_normal_reading(noise_scale=noise_scale)
    abnormal_reading = get_abnormal_reading(noise_scale=noise_scale)
        
    x = []
    y = []
    for _ in range(sample_size):
        isAnomaly = np.random.choice([0, 1], p=[1-abnormal_ratio, abnormal_ratio])
        if isAnomaly == 0:
            i = np.random.randint(0, len(normal_reading)-sequence_length-1)
            x.append(normal_reading[i:i+sequence_length])
        else:
            i = np.random.randint(0, len(abnormal_reading)-sequence_length-1)
            x.append(abnormal_reading[i:i+sequence_length])
        y.append(isAnomaly)
        
    x = np.asarray(x, dtype=np.float32)
    y = np.asarray(y, dtype=np.float32)
    
    return x, y

In [5]:
from keras.layers import Input, Dense, Dropout
from keras.models import Model

# Input a sequence and output the sequence itself
def get_keras_model(sequence_length):
    Input_sequence = Input(shape=(sequence_length, ))
    x = Dense(64, activation='relu')(Input_sequence)
    decoded = Dense(sequence_length)(x)
    
    autoencoder = Model(Input_sequence, output=decoded)
    autoencoder.compile(optimizer='RMSProp', loss='mse')
    
    return autoencoder

Using TensorFlow backend.
/data/anaconda/envs/deeplearning/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/data/anaconda/envs/deeplearning/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/data/anaconda/envs/deeplearning/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/data/anaconda/envs/deeple

In [6]:
import tensorflow as tf

# Input a sequence and output the sequence itself
def get_tf_model(sequence_length):
    tf.reset_default_graph()
    
    Input_sequence = tf.keras.layers.Input(shape=(sequence_length, ), name='input_sequence')
    x = tf.keras.layers.Dense(64, activation='relu')(Input_sequence)
    decoded = tf.keras.layers.Dense(sequence_length, name='output_sequence')(x)
    
    autoencoder = tf.keras.Model(Input_sequence, decoded)
    autoencoder.compile(optimizer='RMSProp', loss='mse')

    return autoencoder

## Train on normal data and test on mixed data

In [7]:
sequence_length = 37

x_train, y_train = get_normal_data(sample_size=2048, sequence_length=sequence_length)
x_test, y_test = get_mixed_data(sequence_length=sequence_length, abnormal_ratio=0.5)

model = get_tf_model(sequence_length=sequence_length)
model.fit(x_train, x_train, batch_size=64, epochs=20, validation_data=(x_test[y_test==0], x_test[y_test==0]))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 2048 samples, validate on 503 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/20
2048/2048 [==============================] - 1s 530us/sample - loss: 0.6080 - val_loss: 0.1774
Epoch 2/20
2048/2048 [==============================] - 0s 31us/sample - loss: 0.0875 - val_loss: 0.0420
Epoch 3/20
2048/2048 [==============================] - 0s 33us/sample - loss: 0.0286 - val_loss: 0.0230
Epoch 4/20
2048/2048 [==============================] - 0s 32us/sample - loss: 0.0183 - val_loss: 0.0180
Epoch 5/20
2048/2048 [==============================] - 0s 32us/sample - loss: 0.0154 - val_loss: 0.0164
Epoch 6/20
2048/2048 [==============================] - 0s 32us/sample - loss: 0.0141 - val_loss: 0.0155
Epoch 7/20
2048/2048 [==============================] - 0s 31us/sample - loss: 0.0132 - val_loss: 0.0145
Epoch 8/20
2048/2048 [===========================

In [8]:
x_pred = model.predict(x_test)

error = [np.sum(np.abs(pred - test)) for (pred, test) in zip(x_pred, x_test)]
error = np.array(error)

error_normal = error[y_test == 0]
error_abnormal = error[y_test == 1]

print(min(error_normal), max(error_normal))
print(min(error_abnormal), max(error_abnormal))

1.6623678 4.5323687
12.582756 25.924002


## Output the model to a .pb file

In [9]:
# reference:
# https://stackoverflow.com/questions/45466020/how-to-export-keras-h5-to-tensorflow-pb
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.compat.v1.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.compat.v1.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = tf.graph_util.convert_variables_to_constants(
            session, input_graph_def, output_names, freeze_var_names)
        return frozen_graph

In [10]:
import tensorflow as tf

sess = tf.compat.v1.keras.backend.get_session()
frozen_graph = freeze_session(sess, output_names=[out.op.name for out in model.outputs])

output_tf_model = 'AnomalyDetection_BaseModel.pb'
# Finally we serialize and dump the output graph to the filesystem
tf.io.write_graph(frozen_graph, "./", output_tf_model, as_text=False)

Instructions for updating:
Use tf.compat.v1.graph_util.convert_variables_to_constants
Instructions for updating:
Use tf.compat.v1.graph_util.extract_sub_graph
INFO:tensorflow:Froze 12 variables.
INFO:tensorflow:Converted 12 variables to const ops.


'./AnomalyDetection_BaseModel.pb'

## Load the model and use it in TensorFlow
Reference: https://leimao.github.io/blog/Save-Load-Inference-From-TF-Frozen-Graph/

In [11]:
tf.reset_default_graph()

graph = tf.Graph()
with tf.gfile.GFile('./' + output_tf_model, 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())

In [12]:
nodes = [n.name + ' => ' +  n.op for n in graph_def.node]
for node in nodes:
    print(node)

input_sequence => Placeholder
dense/kernel => Const
dense/bias => Const
dense/MatMul/ReadVariableOp => Identity
dense/MatMul => MatMul
dense/BiasAdd/ReadVariableOp => Identity
dense/BiasAdd => BiasAdd
dense/Relu => Relu
output_sequence/kernel => Const
output_sequence/bias => Const
output_sequence/MatMul/ReadVariableOp => Identity
output_sequence/MatMul => MatMul
output_sequence/BiasAdd/ReadVariableOp => Identity
output_sequence/BiasAdd => BiasAdd
RMSprop/lr => Const
RMSprop/rho => Const
RMSprop/decay => Const
RMSprop/iterations => Const
training/RMSprop/Variable => Const
training/RMSprop/Variable_1 => Const
training/RMSprop/Variable_2 => Const
training/RMSprop/Variable_3 => Const


In [13]:
with graph.as_default():
    # Define input tensor
    input_tensor = tf.placeholder(np.float32, shape = [None, sequence_length])
    tf.import_graph_def(graph_def, {'input_sequence': input_tensor})
graph.finalize()

In [14]:
layers = [op.name for op in graph.get_operations()]
for layer in layers:
    print(layer)

Placeholder
import/input_sequence
import/dense/kernel
import/dense/bias
import/dense/MatMul/ReadVariableOp
import/dense/MatMul
import/dense/BiasAdd/ReadVariableOp
import/dense/BiasAdd
import/dense/Relu
import/output_sequence/kernel
import/output_sequence/bias
import/output_sequence/MatMul/ReadVariableOp
import/output_sequence/MatMul
import/output_sequence/BiasAdd/ReadVariableOp
import/output_sequence/BiasAdd
import/RMSprop/lr
import/RMSprop/rho
import/RMSprop/decay
import/RMSprop/iterations
import/training/RMSprop/Variable
import/training/RMSprop/Variable_1
import/training/RMSprop/Variable_2
import/training/RMSprop/Variable_3


In [15]:
x_temp, y_temp = get_mixed_data(sample_size = 10, sequence_length=sequence_length, abnormal_ratio=0.5)

sess = tf.Session(graph=graph)
output_tensor = graph.get_tensor_by_name("import/output_sequence/BiasAdd:0")
pred_loaded_model = sess.run(output_tensor, feed_dict={input_tensor: x_temp})

pred_origin_model = model.predict(x_temp)

# The difference should be very close to 0
np.sum(np.abs(pred_loaded_model - pred_origin_model))

0.0